# Import / 라이브러리 호출

In [ ]:
import numpy as np #넘파이 임포트
import random
import os
import pandas as pd #판다스 임포트

In [ ]:
pip install flaml

In [ ]:
# warning 메시지 무시
import warnings
warnings.filterwarnings("ignore")


In [ ]:
# 구글 드라이브 사용 시 
from google.colab import drive
drive.mount('/content/drive')

# Fixed RandomSeed / 랜덤시드 고정

#### 매번 고정된 결과를 얻기 위해서 사용합니다.
#### seed를 고정하지 않는다면 같은 코드라도 매번 다른 결과가 나오게됩니다.
#### 항상 동일한 결과를 얻기 위해서 사용합니다.

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

# Data Load / 데이터 불러오기
#### 먼저 pandas 라이브러리의 read_csv() 함수를 이용해 분석에 사용할 데이터를 불러옵니다.

In [ ]:
# 구글 드라이브 사용 시
if os.getcwd() == '/content':
    work_space = '/content/drive/MyDrive/Colab Notebooks/DACON/236066/'
else:
    work_space = './'

In [ ]:
train = pd.read_csv(work_space + 'train.csv')
test = pd.read_csv(work_space+ 'test.csv')

In [ ]:
train.head()

# Feature & Target Split / 독립변수(X), 종속변수(y)로 나누기

#### 모델을 학습하기 위해서는 독립변수(X)와 종속변수(y)로 나누어야 합니다.
#### 독립변수는 Feature, 종속변수는 Target이라고 불리기도 합니다.
- Feature : 예측을 위해 활용되는 데이터
- Target : 예측하고자 하는 대상

(이 때, 분석에 활용하지 않는 데이터인 'id'는 제거하겠습니다.)

In [ ]:
# X는 독립변수이므로 종속변수를 제거합니다. 또한 target 이외의 분석에 활용하지 않는 데이터(id)를 제거합니다.
train_x = train.drop(columns=['id', 'target', 'snowing'])
# y는 종속변수로 값을 설정합니다.
train_y = train['target']

In [ ]:
# train에서와 마찬가지로 분석에 활용하지 않는 데이터(id)를 제거합니다.
test_x = test.drop(columns=['id', 'snowing'])

# Data Pre-processing

#### 데이터 정규화 : Min-Max Scaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(train_x)
train_x = scaler.transform(train_x)
test_x = scaler.transform(test_x)

# 모델 정의

#### 분석에 사용할 모델을 선언합니다.

In [ ]:
from flaml import AutoML

automl = AutoML()
settings = {
    "time_budget": 18000,  # total running time in seconds
    "metric": 'mae',  # primary metrics for regression can be chosen from: ['mae','mse','r2']
    "estimator_list": ['xgboost'],  # list of ML learners; we tune XGBoost in this example
    "task": 'regression',  # task type
    "seed": 42,  # random seed
    "early_stop": 100,
}
automl.fit(X_train=train_x, y_train=train_y, **settings)

In [ ]:
print('Best hyperparmeter config:', automl.best_config)
print('Best MAE on validation data: {0:.4g}'.format(automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))
print(automl.model.estimator)

In [ ]:
import matplotlib.pyplot as plt

plt.barh(automl.
         feature_names_in_, automl.feature_importances_)

# Prediction / 예측

In [ ]:
pred_automl = automl.predict(test_x)
print('Predicted labels', pred_automl)

# Submit / 제출

In [ ]:
# 제출 파일을 불러옵니다.
submit = pd.read_csv(work_space + 'sample_submission.csv')

In [ ]:
submit['target'] = pred_automl
submit.head()

In [ ]:
submit.to_csv(work_space + 'submit_automl.csv', index=False)